In [1]:
import sys
import os

current_directory = os.getcwd()
path = os.path.dirname(current_directory)
sys.path.append(path)
from Utils import *

%matplotlib widget
from ipywidgets import interact, interactive, widgets
from matplotlib.patches import Rectangle, Circle, Arrow

In [2]:
import glob
import os

import tempfile
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Any, Mapping, Hashable

import monai
from monai.config import print_config
from monai.utils import first
from monai.config import KeysCollection
from monai.data import Dataset, ArrayDataset, create_test_image_3d, DataLoader
from monai.transforms import (
    Transform,
    Compose,
    EnsureChannelFirst,
    LoadImage,
    Orientation,
)
from monaiUtils import MetatensorToSitk, SitkToMetatensor

# Load images
srcdir = os.getcwd()

print('Reading labeled images from: ' + srcdir)
images = sorted(glob.glob(os.path.join(srcdir, 'test_dataset', '*_M.nii.gz')))
labels = sorted(glob.glob(os.path.join(srcdir, 'test_dataset', '*_multi_label.nii.gz')))

# Create dictionary for MONAI
train_files = [
    {'image': image_name, 'label': label_name}
    for image_name, label_name in zip(images, labels)
]
img_file_2d = train_files[0]['image']
label_file_2d = train_files[0]['label']
img_file_3d = train_files[1]['image']
label_file_3d = train_files[1]['label']
fn_keys = ('image', 'label')  # filename keys for image and seg files

# Load original image as SimpleITK object
sitk_input_2d = sitk.ReadImage(images[0], sitk.sitkFloat32)
show_image(sitk_input_2d, 'Loaded 2D sitk image from nii')
sitk_input_3d = sitk.ReadImage(images[1], sitk.sitkFloat32)
show_image(sitk_input_3d, 'Loaded 3D sitk image from nii')

print('SimpleITK object sizes')
print(sitk_input_2d.GetSize())
print(sitk_input_3d.GetSize())


# Get MONAI metatensor from sitk image
metatensor_2d = SitkToMetatensor(sitk_input_2d)
metatensor_3d = SitkToMetatensor(sitk_input_3d)
print('Metatensors for sitk before EnsureChannelFirst')
print(metatensor_2d.shape)
print(metatensor_3d.shape)

# Make it multi-channel
multiChannel = EnsureChannelFirst(channel_dim='no_channel')
metatensor_2d = multiChannel(metatensor_2d)
metatensor_3d = multiChannel(metatensor_3d)
print('Metatensors (sitk) after EnsureChannelFirst')
print(metatensor_2d.shape)
print(metatensor_3d.shape)

# Output images
sitk_output_2d = MetatensorToSitk(metatensor_2d)
show_image(sitk_output_2d, title= 'Image 2D (sitk origin)')
sitk_output_3d = MetatensorToSitk(metatensor_3d)
show_image(sitk_output_3d, title= 'Image 3D (sitk origin)')


# Now load from Nifti instead of using sitk object
loadNifti = LoadImage(image_only=True)
metatensor_2d = loadNifti(img_file_2d)
metatensor_3d = loadNifti(img_file_3d)
print('Metatensors from nifti before EnsureChannelFirst')
print(metatensor_2d.shape)
print(metatensor_3d.shape)

# Make it multi-channel
multiChannel = EnsureChannelFirst(channel_dim='no_channel')
metatensor_2d = multiChannel(metatensor_2d)
metatensor_3d = multiChannel(metatensor_3d)
print('Metatensors (nifti) after EnsureChannelFirst')
print(metatensor_2d.shape)
print(metatensor_3d.shape)

# Output images
sitk_output_2d = MetatensorToSitk(metatensor_2d)
show_image(sitk_output_2d, title= 'Image 2D (nifti origin)')
sitk_output_3d = MetatensorToSitk(metatensor_3d)
show_image(sitk_output_3d, title= 'Image 3D (nifti origin)')


Reading labeled images from: /Users/pl771/Devel/MRINeedleSegmentation/TestingNotebook


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

interactive(children=(IntSlider(value=1, description='z', max=2), Output()), _dom_classes=('widget-interact',)…

SimpleITK object sizes
(192, 192, 1)
(192, 192, 3)
Metatensors for sitk before EnsureChannelFirst
torch.Size([192, 192, 1])
torch.Size([192, 192, 3])
Metatensors (sitk) after EnsureChannelFirst
torch.Size([1, 192, 192, 1])
torch.Size([1, 192, 192, 3])


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

interactive(children=(IntSlider(value=1, description='z', max=2), Output()), _dom_classes=('widget-interact',)…

Metatensors from nifti before EnsureChannelFirst
torch.Size([192, 192, 1])
torch.Size([192, 192, 3])
Metatensors (nifti) after EnsureChannelFirst
torch.Size([1, 192, 192, 1])
torch.Size([1, 192, 192, 3])


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

interactive(children=(IntSlider(value=1, description='z', max=2), Output()), _dom_classes=('widget-interact',)…